## Introduction
Literary authors have a wide variety of styles, contexts, and characters they describe in their works. Many of these styles are repeatable, demonstrating identifiable patterns across their bodies of work. These styles can evolve over time, changing as the lives of the authors themselves evolve. 

## Setup & Initialize Your Resources

Let's start by specifying:

The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region.
The IAM role ARN used to give SageMaker access to your data. It can be fetched using the get_execution_role method from sagemaker python SDK.

In [1]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = sess.default_bucket() # Replace with your own bucket name if needed
print(bucket)
prefix = 'blazingtext/supervised'

arn:aws:iam::023375022819:role/service-role/AmazonSageMaker-ExecutionRole-20181029T121824


INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-023375022819


sagemaker-us-east-1-023375022819


## Data Preperation

The data has been processed from MP3 -> Transcribe -> JSON -> CSV that we want to train the text classification model. BlazingText expects a single preprocessed text file with space separated tokens and each line of the file should contain a single sentence and the corresponding label(s) prefixed by "__label__".

Lets train the text classification model by Jane Austen and Charles Dickens. We will be Training the BlazingText model for supervised text classification.

Need to determine what length of document you use to train the model. This means that you will need to format the Transcribe results into a both data type that BlazingText can read, and a data type that intuitively lends itself well to the author classification problem. We will do this sentence-by-sentence. 

Get the transcribe JSON and convert it to CSV

Get the CSV and break down by sentence per line

## Data Preprocessing

## Training

## Hosting / Inference

## Stop / Close the Endpoint (Optional)